In [4]:
import subprocess
import os

# Get the directory of the current script or the parent directory
script_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.path.dirname(os.getcwd())
print(f'Current working directory = {script_dir}')

def install_requirements(requirements_file):
    try:
        subprocess.check_call(['pip', 'install', '-r', requirements_file])
        print("Successfully installed dependencies from", requirements_file)
    except subprocess.CalledProcessError as e:
        print("Error installing dependencies:", e)

if __name__ == "__main__":
    requirements_file = os.path.join(script_dir, "requirements.txt")
    install_requirements(requirements_file)
    
# Import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from rdflib import Graph, URIRef, Literal, Namespace, RDF
import re
    
# Relative path to the data folder
data_folder = os.path.join(script_dir, 'data', 'ReportRaw')
print(f'Path to data folder = {data_folder}')

# Create and Append a Text File with Data
def txtEdit(text, func, file_path):
    with open(file_path, func) as file:
        file.write(text)

def stress_from_load(l,w,t,mm_in=False):
    if mm_in == True:
        wi = w/25.4
        ti = t/25.4
        area = wi*ti
    else:
        area = w*t
    strs = l/area
    return(strs)

def strain_from_displacement(d,lngth):
    strn = d/lngth
    return(strn)

def true_from_eng(a,b):
    sigma_t = np.copy(a)
    epsilon_t = np.copy(b)
    ii=0
    for ii in range(len(a)):
        sigma_t[ii] = a[ii]*(1+b[ii])
        epsilon_t[ii] = np.log(1+b[ii])
    return(sigma_t,epsilon_t)

results_folder_path = os.path.join(script_dir, 'results')
primary_data_folder = os.path.join(script_dir, 'data', 'PrimaryData')
secondary_data_folder = os.path.join(script_dir, 'data', 'SecondaryData')

# Create Results Folder
if not os.path.exists(results_folder_path):
    os.mkdir(results_folder_path)
print(f'Results folder path = {results_folder_path}')

# Collect all Excel files from subfolders
tensile_files = []
for material_folder in os.listdir(data_folder):
    material_path = os.path.join(data_folder, material_folder)
    if os.path.isdir(material_path):
        excel_files = glob.glob(os.path.join(material_path, '*.xlsx'))
        tensile_files.extend(excel_files)

# Sort the list of file paths
tensile_files = sorted(tensile_files)

# Select only one file for analysis
for tensile_test in tensile_files:
    fname = tensile_test
    material_folder = os.path.basename(os.path.dirname(fname))
    file_name = os.path.splitext(os.path.basename(fname))[0]
    print(f'Selected file for analysis: {fname}')
    print(f'Material: {material_folder}')

    # Create a corresponding folder structure in the results directory
    result_material_folder = os.path.join(results_folder_path, material_folder, file_name)
    if not os.path.exists(result_material_folder):
        os.makedirs(result_material_folder)
    print(f'Metadata Result Folder = {result_material_folder}')

    # Path to the results file
    metadata_file_path = os.path.join(result_material_folder, 'results.txt')
    
    # Create a results file inside the result folder using txtEdit function
    txtEdit(f'Results for {file_name} in {material_folder}\n\n', 'w', metadata_file_path)
    print(f'Results file created at: {metadata_file_path}')

    # Read the selected Excel file
    try:
        df = pd.read_excel(fname, sheet_name='Sheet2', header=None)
        print("Successfully read the Excel file.")
    except Exception as e:
        print("Error occurred while reading the Excel file:", e)
        txtEdit('Data Name Convention Breached\n\n', 'a', metadata_file_path)
    else:
        print("No Excel files found in the specified directory.")
        

    #Find Column Index of mm
    dfcol = df
    col = dfcol
    for i in range(len(col)):
        try:
            if any(dfcol.iloc[i].str.contains('mm')==True):
                breakpoint = i
                break
        #Uniformalize? the Data
        except(AttributeError):
            dfcol = dfcol.T
            dfcol.insert(0,'0',np.nan)
            dfcol = dfcol.T
            dfcol.fillna('empty',inplace=True)
            if any(dfcol.iloc[i].str.contains('mm')==True):
                breakpoint = i
                break
            
    # Save raw data to a DataFrame
    raw_data_df = dfcol.copy() 
    
    # Drop the first row as it contains the headers
    raw_data_df = raw_data_df.iloc[1:]
    
    # Create the folder structure if it doesn't exist
    target_folder = os.path.join(primary_data_folder, material_folder)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    # Define the full path for the CSV file
    csv_file_path = os.path.join(target_folder, file_name)
    
    # Save the raw load-displacement data to CSV
    raw_data_df.to_csv(csv_file_path + ".csv", index=False, header = ["Displacement (mm)", "Load (N)"])
    
    print(f"Raw Data Saved to = {csv_file_path}")
    
    try:
        df2 = pd.read_excel(fname, sheet_name='Test Report')
    except:
        print('Data Name Convention Breached\n\n')
        txtEdit('Data Name Convention Breached\n\n', 'a', metadata_file_path)
        #continue
        
    #Find Length
    a = np.where(df2['Default Test Run Report'].str.contains('Initial Marked Gage Length')==True)
    a = int(a[0])
    dflen = pd.read_excel(fname, sheet_name='Test Report',skiprows=a,nrows=1)
    dflen = dflen.T
    s_length = dflen.iloc[1,0]
    print('Length:'+str(s_length))
    txtEdit('Length:'+str(s_length)+'\n','a', metadata_file_path)
        
    #Find Width
    a = np.array(np.where(df2['Default Test Run Report'].str.contains('Width')==True)).flatten()
    a = int(a[0])
    dfw = pd.read_excel(fname, sheet_name='Test Report',skiprows=a,nrows=1)
    dfw = dfw.T
    width = dfw.iloc[1,0]
    print('Width:'+str(width))
    txtEdit('Width:'+str(width)+'\n','a', metadata_file_path)
        
    #Find Thickness
    a = np.array(np.where(df2['Default Test Run Report'].str.contains('Thickness')==True)).flatten()
    a = int(a[0])
    dft = pd.read_excel(fname, sheet_name='Test Report',skiprows=a,nrows=1)
    dft = dft.T
    thickness = dft.iloc[1,0]
    print('Thickness:'+str(thickness))
    txtEdit('Thickness:'+str(thickness)+'\n','a', metadata_file_path)
        
    #Find Stress and Strain
    try:
        dfdat = pd.read_excel(fname, sheet_name='Sheet2',skiprows=breakpoint)
        dfdat.dropna(subset=['mm'],inplace=True)
        completion = True
    except:
        completion = False
     
        if completion == False:
            try:
                dfdat = pd.read_excel(fname, sheet_name='Sheet2',skiprows=breakpoint+1)
                dfdat.dropna(subset=['mm'],inplace=True)
            except:
                print('Data Layout Unsupported\n\n')
                txtEdit('Data Layout Unsupported\n\n','a', metadata_file_path)
                #continue
            
    try:
        dfdat.dropna(subset=['N'],inplace=True)
        load = np.array(dfdat['N'])
        stress = stress_from_load(load,width,thickness)
        y_units = 'MPa'
        print('Units: MPa')
        txtEdit('Units: MPa'+'\n','a', metadata_file_path)
    except:
        dfdat.dropna(subset=['lbf'],inplace=True)
        load = np.array(dfdat['lbf'])
        stress = stress_from_load(load,width,thickness,mm_in=True)
        y_units = 'psi'
        print('Units: psi')
        txtEdit('Units: psi'+'\n','a', metadata_file_path)
        
    displacement = np.array(dfdat['mm'])
    strain = strain_from_displacement(displacement,s_length)


    """
    Smooth Dataset
    """
    
    #Set Moving Window
    winSize = 25
        
    smooth_x = np.array([])
    smooth_y = np.array([])
    length = len(stress)
    j=0
    while j < length - 1:
        #print("j:", j, "length:", length)  # Debugging print statement
        check = np.sign(stress[j])
        try:
            strain[j+1]
        except:
            if check < 0:
                stress = np.delete(stress, j)
                strain = np.delete(strain, j)
                break
            
        if check < 0:
            stress = np.delete(stress, j)
            strain = np.delete(strain, j)
            j = j - 1
        elif np.sign(strain[j+1] - strain[j]) < 0:
            stress = np.delete(stress, j)
            strain = np.delete(strain, j)
            j = j - 1
        else:
            pass
        length = len(stress)
        j = j + 1
     #   print(stress)
        
    i = 0
    while i < len(strain):
        datx = np.average(strain[i:i+winSize])
        daty = np.average(stress[i:i+winSize])
        datarrX = np.array([datx])
        datarrY = np.array([daty])
        smooth_x = np.append(smooth_x,datarrX)
        smooth_y = np.append(smooth_y,datarrY)
        i = i+winSize
        
    # Create DataFrames for processed stress-strain data
    processed_stress_strain_df = pd.DataFrame({'strain': smooth_x, 'stress': smooth_y})
    
    # Create the folder structure if it doesn't exist
    target_folder = os.path.join(secondary_data_folder, material_folder)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    # Define the full path for the CSV file
    csv_file_path = os.path.join(target_folder, file_name)
    
    # Save the raw load-displacement data to CSV
    processed_stress_strain_df.to_csv(csv_file_path + ".csv", index=False, header = ["Strain", "Stress (MPa)"])
    
    print(f"Processed Data Saved to = {csv_file_path}")
    
    #print(smooth_x,smooth_y)
    fig,ax = plt.subplots(figsize=(10,10))
    figname = str(material_folder) + ' Stress-Strain Graph'
    plt.title(figname)
    plt.ylabel('Stress ('+y_units+')')
    plt.xlabel("Strain")
    plt.plot(smooth_x,smooth_y,marker='o',color='black')
    fig.savefig(os.path.join(result_material_folder, figname) + '.png', dpi=300, bbox_inches='tight')
    plt.close()

    """
    True Stress/Strain
    """
    #Call function
    true_smooth_y,true_smooth_x = true_from_eng(smooth_y,smooth_x)
    
    # Create DataFrames for processed stress-strain data
    true_stress_strain_df = pd.DataFrame({'strain': true_smooth_x, 'stress': true_smooth_y})
       
    #Plot True Stress/Strain
    fig,ax = plt.subplots(figsize=(10,10))
    figname = str(material_folder)+' True Stress-Strain Graph'
    plt.title(figname)
    plt.ylabel('Stress ('+y_units+')')
    plt.xlabel("Strain")
    plt.plot(smooth_x,smooth_y,marker='o',color='black')
    plt.plot(true_smooth_x,true_smooth_y,marker='s',color='purple')
    fig.savefig(os.path.join(result_material_folder, figname) + '.png', dpi=300, bbox_inches='tight')
    plt.close()
    smooth_y = true_smooth_y
    smooth_x = true_smooth_x

    """
    Young's Modulus
    """
    #Find E
    start=10
    i=start+1
    Ei = np.array([])
    Eact = np.array([])
    error = 0.1
    pings = 0
    k=1
    while i < len(smooth_x):
        Epoint = (smooth_y[i]-smooth_y[i-1])/(smooth_x[i]-smooth_x[i-1])
        Earr = np.array([Epoint])
        Ei = np.append(Ei,Earr)
        Eavg = np.average(Ei)
        if i >= start+2:
            if Eavg/Ei[k-1] > (3+error) or Eavg/Ei[k-1] < (1-error):
       #       print(Ei[k-1])
                pings = pings+1
            else:
                Eact = np.append(Eact,Ei[k-1])
                pings = 0
                breakpoint = i
        else:
            pass
        
        if pings >= 5:
            Ei = np.delete(Ei,k-1)
            break
        else:
            pass
        i = i + 1
        k = k + 1
      #  print(Eact)
    
    E = np.average(Eact)
    print("Young's Modulus = {}".format(int(np.round(E,0)))+' '+ y_units)
    txtEdit("Young's Modulus = {}".format(int(np.round(E,0)))+' '+y_units+'\n','a', metadata_file_path)
  #  print(breakpoint)

    """
    Proportional Limit
    """
    #Find Proportional Limit
    length = winSize * breakpoint
    winMin = length - winSize
    winMax = length + winSize
    small_x = np.array([])
    small_y = np.array([])
    sEi = np.array([])
        
    winSizeSmall = 1
        
    i = winMin
    smallError = 0.1
    k = 1
    while i < winMax:
        pl = i
        sdatx = np.average(strain[i:i+winSizeSmall])
        sdaty = np.average(stress[i:i+winSizeSmall])
        sdatarrX = np.array([sdatx])
        sdatarrY = np.array([sdaty])
        small_x = np.append(small_x,sdatarrX)
        small_y = np.append(small_y,sdatarrY)
        if len(small_x) >= 2:
            sEpoint = (small_y[k-1]-small_y[k-2])/(small_x[k-1]-small_x[k-2])
            sEarr = np.array([sEpoint])
            sEi = np.append(sEi,sEarr)
            if len(sEi) >= 2:
                if sEi[k-3]/sEi[k-2] > (1+smallError) or sEi[k-3]/sEi[k-2] < (1-smallError):
                    break
                else:
                    pass
            else:
                pass
        else:
            pass
        k = k+1
        i = i+winSizeSmall
        
    plx = np.linspace(strain[pl],strain[pl],2)
    ply = np.linspace(0,stress[pl],2)
    
    print("Proportional Limit = {:.2f} {}".format(ply[-1], y_units))
    txtEdit("Proportional Limit = {:.2f} {}\n".format(ply[-1], y_units), 'a', metadata_file_path)


    # Plot proportional limit
    fig, ax = plt.subplots(figsize=(10, 10))
    figname = f"{material_folder} Proportional Limit Graph"
    plt.title(figname)
    plt.ylabel('Stress (' + y_units + ')')
    plt.xlabel("Strain")
    plt.plot(smooth_x, smooth_y, marker='o', color='black')
    plt.plot(plx, ply, marker='o', color='red', linestyle='dashed', label='Proportional Limit')
    plt.legend()
    plt.text(plx[-1], ply[-1], f'  Proportional Limit: {ply[-1]:.2f} {y_units}', 
             verticalalignment='bottom', horizontalalignment='left')
    
    # Save and display the plot
    fig.savefig(os.path.join(result_material_folder, figname) + '.png', dpi=300, bbox_inches='tight')
    plt.close()

    """
    0.2% Offset
    """
    #Find 0.2% Offset
    stress_offset = np.copy(smooth_y)
    strain_offset = np.copy(smooth_x)
    for i in range(len(stress_offset)):
        strain_offset[i] = (stress_offset[i]/E) + 0.002
    indx = int(np.argwhere(np.diff(np.sign(strain_offset-smooth_x))).flatten())
      #  print(indx)
    mult = 2
    length = winSize * indx
      #  print(length)
    winMin = length - winSize*mult
    winMax = length + winSize*mult
    small_strain = np.array([])
    small_stress = np.array([])
    i = winMin
    while i < winMax:
        offset_datx = np.average(strain[i:i+winSizeSmall])
        offset_daty = np.average(stress[i:i+winSizeSmall])
        offset_datarrX = np.array([offset_datx])
        offset_datarrY = np.array([offset_daty])
        small_strain = np.append(small_strain,offset_datarrX)
        small_stress = np.append(small_stress,offset_datarrY)
        i = i+winSizeSmall
        
    stress_offset_fine = np.copy(small_stress)
    strain_offset_fine = np.copy(small_strain)
    
    for i in range(len(stress_offset_fine)):
        strain_offset_fine[i] = (stress_offset_fine[i]/E) + 0.002
        
    window=winMax-winMin
    
    ###USE ONLY FOR LARGE DATA SETS
    fine_indx = int(np.argwhere(np.diff(np.sign(strain_offset_fine-small_strain))).flatten())
      #  print(fine_indx)
    new_indx = winMin+((fine_indx)*winSizeSmall)
      # print(new_indx)
      
    print('0.2% Yield Strength = {}'.format(np.round(stress_offset_fine[fine_indx],3)) + y_units)
    txtEdit('0.2% Yield Strength = {}'.format(np.round(stress_offset_fine[fine_indx],3))+y_units+'\n','a', metadata_file_path)

    """
    Modulus of Resilience
    """
    ur = (1/2)*(stress[pl]**2/E)
    print('Modulus of Resilience at Proportional Limit = {}'.format(np.round(ur,3))+" "+y_units)
    txtEdit('Modulus of Resilience at Proportional Limit = {}'.format(np.round(ur,3))+" "+y_units+'\n','a', metadata_file_path)
    ur_offset = (1/2)*(stress_offset_fine[fine_indx]**2/E)
    print('Strain Energy Density at 0.2% Offset = {}'.format(np.round(ur_offset,3))+" "+ y_units)
    txtEdit('Strain Energy Density at 0.2% Offset = {}'.format(np.round(ur_offset,3))+" "+y_units+'\n','a', metadata_file_path)
        
    #Plot Moduli of Resilience
    ofst2x = np.linspace(strain_offset_fine[fine_indx],strain_offset_fine[fine_indx],2)
    ofst2y = np.linspace(0,stress_offset_fine[fine_indx],2)
    fig,ax = plt.subplots(figsize=(10,10))
    figname = str(material_folder)+' Proportional Limit and 0.2% Yield Strength'
    plt.title(figname)
    plt.xlim(0,strain[new_indx+winSize])
    plt.ylim(0,stress[new_indx+winSize])
    plt.ylabel('Stress ('+y_units+')')
    plt.xlabel("Strain")
    plt.plot(strain_offset,stress_offset,color='b',linestyle='-.')
    plt.plot(smooth_x,smooth_y,marker='o',color='black')
    plt.plot(plx,ply,marker='^',color='black',linestyle='--')
    plt.plot(ofst2x,ofst2y,color='black',linestyle='--')
    plt.plot(strain_offset_fine[fine_indx],stress_offset_fine[fine_indx],marker='<',color='b',markersize=10)
    plt.annotate('pl=({},{})'.format(np.round(strain[pl],4),np.round(stress[pl],1)),
                    (strain[pl],stress[pl]),(strain[pl]+0.0001,stress[pl]))
    plt.annotate('0.2%=({},{})'.format(np.round(strain_offset_fine[fine_indx],3),
                                           np.round(stress_offset_fine[fine_indx],1)),
                     (strain_offset_fine[fine_indx],stress_offset_fine[fine_indx]),
                     (strain_offset_fine[fine_indx]+0.0001,stress_offset_fine[fine_indx]-5))
        
    plt.fill_between(strain[0:new_indx],stress[0:new_indx],color='darkorange',alpha=0.7,hatch='/',
                         label='u={}'.format(np.round(ur_offset,3)))
    plt.fill_between(strain[0:pl],stress[0:pl],color='royalblue',alpha=0.7,hatch='|',
                        label='ur={}'.format(np.round(ur,3)))
    plt.legend()
    fig.savefig(os.path.join(result_material_folder, figname) + '.png', dpi=300, bbox_inches='tight')
    plt.close()

        #Plot Moduli of Resilience
    ofst2x = np.linspace(strain_offset_fine[fine_indx],strain_offset_fine[fine_indx],2)
    ofst2y = np.linspace(0,stress_offset_fine[fine_indx],2)
    fig,ax = plt.subplots(figsize=(10,10))
    figname = str(material_folder)+' Proportional Limit and 0.2% Yield Strength'
    plt.title(figname)
    #plt.xlim(0,strain[new_indx+winSize])
    #plt.ylim(0,stress[new_indx+winSize])
    plt.ylabel('Stress ('+y_units+')')
    plt.xlabel("Strain")
    plt.plot(strain_offset,stress_offset,color='b',linestyle='-.')
    plt.plot(smooth_x,smooth_y,marker='o',color='black')
    plt.plot(plx,ply,marker='^',color='black',linestyle='--')
    plt.plot(ofst2x,ofst2y,color='black',linestyle='--')
    plt.plot(strain_offset_fine[fine_indx],stress_offset_fine[fine_indx],marker='<',color='b',markersize=10)
    plt.annotate('pl=({},{})'.format(np.round(strain[pl],4),np.round(stress[pl],1)),
                    (strain[pl],stress[pl]),(strain[pl]+0.0001,stress[pl]))
    plt.annotate('0.2%=({},{})'.format(np.round(strain_offset_fine[fine_indx],3),
                                           np.round(stress_offset_fine[fine_indx],1)),
                     (strain_offset_fine[fine_indx],stress_offset_fine[fine_indx]),
                     (strain_offset_fine[fine_indx]+0.0001,stress_offset_fine[fine_indx]-5))
        
    plt.fill_between(strain[0:new_indx],stress[0:new_indx],color='darkorange',alpha=0.7,hatch='/',
                         label='u={}'.format(np.round(ur_offset,3)))
    plt.fill_between(strain[0:pl],stress[0:pl],color='royalblue',alpha=0.7,hatch='|',
                        label='ur={}'.format(np.round(ur,3)))
    plt.legend()
    fig.savefig(os.path.join(result_material_folder, figname) + '.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    """
    Modulus of Toughness
    """
    #Create Rectangles
    area = 0
    for i in range(1,len(smooth_x)):
        w = smooth_x[i]-smooth_x[i-1]
        h = (smooth_y[i]+smooth_y[i-1])/2
        rec = w*h
        area=area+rec
    print('Modulus of Toughness = {}'.format(np.round(area,3))+" "+y_units)
    txtEdit('Modulus of Toughness = {}'.format(np.round(area,3))+" "+y_units+'\n','a', metadata_file_path)

    """
    UTS and Strain at Failure
    """
    #Ultimate Tensile Strength
    UTS = np.argmax(smooth_y)
    #UTS = smooth_y[UTS]
    print('UTS={}'.format(np.round(smooth_y[UTS],1))+" "+y_units)
    txtEdit('UTS={}'.format(np.round(smooth_y[UTS],1))+" "+y_units+'\n','a', metadata_file_path)
        
    #Strain at Failure
    ef = np.argmax(smooth_x)
    print('ef={}'.format(np.round(smooth_x[ef],4)))
    txtEdit('ef={}'.format(np.round(smooth_x[ef],4))+'\n','a', metadata_file_path)
    xef=np.linspace(smooth_x[ef],smooth_x[ef],2)
    yef=np.linspace(0,smooth_y[ef],2)
        
    #Find Elastic Recovery
    tantheta = stress[pl]/strain[pl]
    theta = np.arctan(tantheta)
    elastic_ef = smooth_x[ef]-(smooth_y[ef]/np.tan(theta))
    print('er={}'.format(np.round(elastic_ef,4)))
    txtEdit('er={}'.format(np.round(elastic_ef,4))+'\n','a', metadata_file_path)
    xEAF = np.linspace(elastic_ef,smooth_x[ef],2)
    yEAF = np.linspace(0,smooth_y[ef],2)
        
    #Plot UTS and Strain at Failure
    fig,ax = plt.subplots(figsize=(10,10))
    figname = str(material_folder)
    plt.title(figname)
    plt.ylabel('Stress ('+y_units+')')
    plt.xlabel("Strain")
    plt.plot(smooth_x,smooth_y,marker='o',color='black')
        
    #Proportional Limit
    plt.plot(plx,ply,marker='^',color='black',linestyle='--')
    plt.annotate('pl=({},{})'.format(np.round(strain[pl],4),np.round(stress[pl],1)),
                     (strain[pl],stress[pl]),(strain[pl]+0.0001,stress[pl]))
        
    #0.2% Offset
    plt.plot(strain_offset,stress_offset,color='b',linestyle='-.')
    plt.annotate('0.2%=({},{})'.format(np.round(strain_offset_fine[fine_indx],3),
                                           np.round(stress_offset_fine[fine_indx],1)),
                     (strain_offset_fine[fine_indx],stress_offset_fine[fine_indx]),
                     (strain_offset_fine[fine_indx]+0.005,stress_offset_fine[fine_indx]))
    plt.plot(strain[new_indx],stress[new_indx],marker='<',color='b',markersize=10)
        
    #Resilience/Elastic Strain Energy/Toughness
    plt.plot(ofst2x,ofst2y,color='black',linestyle='--')
    plt.fill_between(smooth_x,smooth_y,color='green',alpha=0.7,hatch='-',
                        label='ut={}'.format(np.round(area,3)))
    plt.fill_between(strain[0:new_indx],stress[0:new_indx],color='darkorange',alpha=0.7,hatch='/',
                         label='u={}'.format(np.round(ur_offset,3)))
    plt.fill_between(strain[0:pl],stress[0:pl],color='royalblue',alpha=0.7,hatch='|',
                        label='ur={}'.format(np.round(ur,3)))
        
    #UTS and Strain at Failure
    plt.plot(smooth_x[UTS],smooth_y[UTS],marker='o',markersize=10,color='magenta')
    plt.annotate('UTS=({},{})'.format(np.round(smooth_x[UTS],3),np.round(smooth_y[UTS],1)),
                     (smooth_x[UTS],smooth_y[UTS]),(smooth_x[UTS]-0.002,smooth_y[UTS]+5))
    plt.plot(xef,yef,color='black',linestyle='--')
    plt.plot(smooth_x[ef],smooth_y[ef],marker='o',color='black')
    plt.annotate('ef=({},{})'.format(np.round(smooth_x[ef],4),np.round(smooth_y[ef],1)),
                     (smooth_x[ef],smooth_y[ef]),(smooth_x[ef]-0.055,smooth_y[ef]-10))
        
    plt.plot(xEAF,yEAF,color='blue',linestyle='-.')
    plt.plot(xEAF[0],yEAF[0],marker='o',color='b')
    plt.annotate('er=({},{})'.format(np.round(xEAF[0],4),np.round(yEAF[0],1)),
                     (xEAF[0],yEAF[0]),(xEAF[0]-0.055,yEAF[0]-10))
        
        
    plt.legend()
    fig.savefig(os.path.join(result_material_folder, figname) + '.png', dpi=300, bbox_inches='tight')
    plt.close()
    print('\n\n')

    import rdflib

    # Create an RDF graph for the experimental data
    data_graph = rdflib.Graph()
    
    # Add namespaces
    TTO = rdflib.Namespace("https://w3id.org/pmd/tto/")
    CORE = rdflib.Namespace("https://w3id.org/pmd/co/")
    data_graph.bind("tto", TTO)
    data_graph.bind("core", CORE)
    
    # Define the tensile test instance
    test_instance = "https://github.com/Wino1301/MaRCN_Tensile_Ontology/tree/main/data/ReportRaw/" + material_folder + "/" + file_name
    test_instance = rdflib.URIRef(test_instance)
    
    # Add the tensile test instance to the graph
    data_graph.add((test_instance, rdflib.RDF.type, TTO.TensileTest))
    
    # Function to add property with unit as a blank node
    def add_property_with_unit(graph, instance, property_uri, value, unit):
        value_node = rdflib.BNode()
        graph.add((instance, property_uri, value_node))
        graph.add((value_node, rdflib.RDF.type, CORE.ValueObject))
        graph.add((value_node, rdflib.RDF.value, rdflib.Literal(value, datatype=rdflib.XSD.float)))
        graph.add((value_node, CORE.unit, rdflib.URIRef(unit)))
    
    # Add properties with units to the tensile test instance
    add_property_with_unit(data_graph, test_instance, TTO.OriginalGaugeLength, float(s_length), "https://w3id.org/pmd/co/unit#mm")
    add_property_with_unit(data_graph, test_instance, TTO.OriginalWidth, width, "https://w3id.org/pmd/co/unit#mm")
    add_property_with_unit(data_graph, test_instance, TTO.OriginalThickness, thickness, "https://w3id.org/pmd/co/unit#mm")
    add_property_with_unit(data_graph, test_instance, TTO.YoungsModulus, float(np.round(E,0)), "https://w3id.org/pmd/co/unit#MPa")
    add_property_with_unit(data_graph, test_instance, TTO.YieldStrength, np.round(stress_offset_fine[fine_indx],3), "https://w3id.org/pmd/co/unit#MPa")
    add_property_with_unit(data_graph, test_instance, TTO.UltimateTensileStrength, np.round(smooth_y[UTS],1), "https://w3id.org/pmd/co/unit#MPa")
    add_property_with_unit(data_graph, test_instance, TTO.PercentageExtension, np.round(smooth_x[ef],4), "https://w3id.org/pmd/co/unit#percent")
    
    # Add PrimaryData, SecondaryData, and MaterialDesignation as strings
    PrimaryData_link = "https://github.com/Wino1301/MaRCN_Tensile_Ontology/tree/main/data/PrimaryData/"+ material_folder + "/" + file_name + ".csv"
    SecondaryData_link = "https://github.com/Wino1301/MaRCN_Tensile_Ontology/tree/main/data/SecondaryData/"+ material_folder + "/" + file_name + ".csv"
    data_graph.add((test_instance, CORE.PrimaryData, rdflib.Literal(PrimaryData_link, datatype=rdflib.XSD.string)))
    data_graph.add((test_instance, CORE.SecondaryData, rdflib.Literal(SecondaryData_link, datatype=rdflib.XSD.string)))
    data_graph.add((test_instance, CORE.MaterialDesignation, rdflib.Literal(material_folder, datatype=rdflib.XSD.string)))
    
    # Serialize the RDF data to a TTL file
    output_file =os.path.join(result_material_folder, file_name + ".ttl") 
    
    data_graph.serialize(destination=output_file, format="turtle")
    
    print(f"Experimental RDF data serialized to {output_file}")

Current working directory = D:\FAIR_Ontology\MaRCN_Tensile_Ontology
Path to data folder = D:\FAIR_Ontology\MaRCN_Tensile_Ontology\data\ReportRaw
Results folder path = D:\FAIR_Ontology\MaRCN_Tensile_Ontology\results
Selected file for analysis: D:\FAIR_Ontology\MaRCN_Tensile_Ontology\data\ReportRaw\Aluminium_6061\Tensile_001.xlsx
Material: Aluminium_6061
Metadata Result Folder = D:\FAIR_Ontology\MaRCN_Tensile_Ontology\results\Aluminium_6061\Tensile_001
Results file created at: D:\FAIR_Ontology\MaRCN_Tensile_Ontology\results\Aluminium_6061\Tensile_001\results.txt
Successfully read the Excel file.
No Excel files found in the specified directory.
Raw Data Saved to = D:\FAIR_Ontology\MaRCN_Tensile_Ontology\data\PrimaryData\Aluminium_6061\Tensile_001
Length:50
Width:13.49
Thickness:3.09
Units: MPa
Processed Data Saved to = D:\FAIR_Ontology\MaRCN_Tensile_Ontology\data\SecondaryData\Aluminium_6061\Tensile_001
Young's Modulus = 58884 MPa
Proportional Limit = 158.21 MPa
0.2% Yield Strength = 187.